In [ ]:
import importlib
import subprocess
import sys
import gc

def check_and_install_package(package_name, version=None):
    try:
        importlib.import_module(package_name)
        print(f"\n{package_name} is already installed.")
    except ImportError:
        print(f"\n{package_name} is NOT installed. Installing now...")
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package_name}=={version}"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

# List of packages to check along with specific versions if necessary
packages = [
    {"name": "tqdm", "version": None},
    {"name": "pyspark", "version": "3.5.2"},
    {"name": "gdown", "version": None},
    {"name": "numpy", "version": "1.23.5"}
]

# Checking and installing the packages
for package in packages:
    check_and_install_package(package["name"], package["version"])


tqdm is already installed.

pyspark is already installed.

gdown is already installed.

numpy is already installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LinearRegressor") \
    .config("spark.driver.memory", "40g") \
    .config("spark.executor.memory", "40g") \
    .config("spark.driver.maxResultSize", "5g") \
    .config("spark.executor.memoryOverhead", "8g") \
    .config("spark.executor.cores", "5") \
    .config("spark.kryoserializer.buffer.max", "2047m") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThreads=4 -XX:ParallelGCThreads=4") \
    .getOrCreate()

# Verifying Spark session creation
print(f"Spark session started with version: {spark.version}")


Spark session started with version: 3.5.3


# **Predictions BEFORE Feature Engineering**

In [ ]:
# loading the df

!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Processed_DF.parquet' /content/

output_path = '/content/Processed_DF.parquet'
df = spark.read.parquet(output_path)
print("The Processed DataFrame has been loaded successfully.")


The Processed DataFrame has been loaded successfully.


In [ ]:
df = df.repartition(100)  # Repartitioning into 100 partitions for parallelism

In [ ]:
# Printing the shape of the DataFrame
total_rows = df.count()
total_columns = len(df.columns)

print(f"The shape of the loaded DataFrame is: ({total_rows}, {total_columns})")

The shape of the loaded DataFrame is: (3000040, 42)


In [ ]:
from functools import reduce
from pyspark.sql.functions import col

# Checking for rows with at least one null value in any column
rows_with_null = df.filter(
    reduce(lambda x, y: x | y, (col(c).isNull() for c in df.columns))
).count()

print(f"Number of rows with at least one null value: {rows_with_null}")

Number of rows with at least one null value: 0


### **Handling Categorical Coloumns**

In [ ]:
df=df.drop('description','major_options','mileage')
# Keeping the columns ['exterior_color','dealer_zip','interior_color']

In [ ]:
# Counting unique values in 'exterior_color' and 'interior_color' columns
exterior_colors_count = df.select('exterior_color').distinct().count()
interior_colors_count = df.select('interior_color').distinct().count()

print(f"Unique exterior colors: {exterior_colors_count}")
print(f"Unique interior colors: {interior_colors_count}")

Unique exterior colors: 23036
Unique interior colors: 38528


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType

# Listing top colors for both exterior and interior
colors = ['White', 'Black', 'Gray', 'Silver', 'Red', 'Blue', 'Brown', 'Green', 'Beige', 'Orange', 'Gold', 'Yellow', 'Purple']

# Creating a UDF to find colors in the color columns
@F.udf(returnType=ArrayType(StringType()))
def find_colors(color_string):
    if color_string is None or color_string.strip() == "":
        return ["Other"]  # Handle empty or null values
    found_colors = [c for c in colors if c.lower() in color_string.lower()]
    return found_colors if found_colors else ["Other"]  # Label non-matching colors as "Other"

# Applying the UDF to both the exterior and interior color columns
df = df.withColumn("exterior_color_array", find_colors("exterior_color"))
df = df.withColumn("interior_color_array", find_colors("interior_color"))

# Creating a column with the count of colors found for both exterior and interior
df = df.withColumn("exterior_color_count", F.size("exterior_color_array"))
df = df.withColumn("interior_color_count", F.size("interior_color_array"))

# Joining the color arrays into string columns
df = df.withColumn("exterior_color", F.array_join("exterior_color_array", ", "))
df = df.withColumn("interior_color", F.array_join("interior_color_array", ", "))

# Labeling mixed colors for both exterior and interior colors
df = df.withColumn(
    "exterior_color",
    F.when(F.col("exterior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("exterior_color")))

df = df.withColumn(
    "interior_color",
    F.when(F.col("interior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("interior_color")))

# Dropping temporary columns
df = df.drop("exterior_color_array", "exterior_color_count", "interior_color_array", "interior_color_count")


In [ ]:
# Counting the occurrences of each exterior and interior color and calculating percentages
exterior_color_counts = df.groupBy("exterior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))

interior_color_counts = df.groupBy("interior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))

# Showing the results
print("Exterior Color Distribution:")
exterior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Exterior Color Distribution:
+--------------+------+----------+
|exterior_color|count |percentage|
+--------------+------+----------+
|White         |675979|22.53     |
|Black         |580148|19.34     |
|Other         |543638|18.12     |
|Silver        |384540|12.82     |
|Blue          |253263|8.44      |
|Red           |242331|8.08      |
|Gray          |231172|7.71      |
|Green         |23026 |0.77      |
|Mixed Colors  |19728 |0.66      |
|Brown         |12905 |0.43      |
|Orange        |11638 |0.39      |
|Gold          |10544 |0.35      |
|Beige         |5065  |0.17      |
|Yellow        |4855  |0.16      |
|Purple        |1208  |0.04      |
+--------------+------+----------+



In [ ]:
print("Interior Color Distribution:")
interior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Interior Color Distribution:
+--------------+-------+----------+
|interior_color|count  |percentage|
+--------------+-------+----------+
|Black         |1624033|54.13     |
|Other         |577578 |19.25     |
|Gray          |383966 |12.8      |
|Mixed Colors  |171212 |5.71      |
|White         |91545  |3.05      |
|Brown         |65943  |2.2       |
|Red           |34117  |1.14      |
|Silver        |24124  |0.8       |
|Blue          |22828  |0.76      |
|Green         |2048   |0.07      |
|Gold          |1193   |0.04      |
|Orange        |1133   |0.04      |
|Yellow        |134    |0.0       |
|Purple        |121    |0.0       |
|Beige         |65     |0.0       |
+--------------+-------+----------+



In [ ]:
print(f"Final processed DataFrame used for the model has {df.count()} rows and {len(df.columns)} columns.")

Final processed DataFrame used for the model has 3000040 rows and 39 columns.


In [ ]:
# Calculating the average price
avg_price = df.agg({"price": "avg"}).collect()[0][0]
print(f"Average price of a car: {round(avg_price)}")

Average price of a car: 29933


In [ ]:
import pandas as pd
from IPython.display import display
import pyspark.sql.functions as F

# Converting the Spark DataFrame to a Pandas DataFrame and displaying 5 random rows with all columns
pd.set_option('display.max_columns', None)
pandas_df = df.orderBy(F.rand()).limit(5).toPandas()
display(pandas_df)


,fuel_type,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,fuel_tank_volume,height,highway_fuel_economy,horsepower,interior_color,is_new,latitude,length,listed_date,listing_color,longitude,make_name,maximum_seating,model_name,price,savings_amount,seller_rating,sp_name,torque,transmission,transmission_display,wheel_system_display,wheelbase,width,year,combined_fuel_economy,legroom,log_mileage,major_options_count
0,Gasoline,SUV / Crossover,Davison,26.0,71,48423,1500.0,I4,Gray,True,14.9,65.40,30.0,170.0,Black,True,43.015999,182.3,2020-07-01,GRAY,-83.517502,GMC,5.0,Terrain,22662.0,0,4.000000,Todd Wenzel Buick GMC of Davison,265.22,A,Automatic,Front-Wheel Drive,107.3,72.4,2020,28.0,80.60,8.43,8
1,Electric,Wagon,Arlington,127.0,34,02476,2500.0,Electric Motor,Orange,True,0.0,65.87,108.0,241.0,Gray,True,42.422298,189.8,2020-08-06,ORANGE,-71.173203,Chevrolet,5.0,Bolt EV,34968.0,0,4.000000,Mirak Chevrolet,265.22,A,Automatic,Front-Wheel Drive,111.0,77.2,2020,117.5,80.16,1.39,4
2,Gasoline,SUV / Crossover,West Palm Beach,21.0,2,33415,3200.0,V6,White,False,15.9,65.70,29.0,271.0,Black,False,26.681499,182.0,2020-09-08,WHITE,-80.112099,Jeep,5.0,Cherokee,16800.0,937,4.230769,Port Motors,239.00,A,9-Speed Automatic,Front-Wheel Drive,106.3,73.2,2016,25.0,81.40,10.79,10
3,Gasoline,SUV / Crossover,Glenview,15.0,43,60025,4600.0,V8,Black,True,23.0,74.20,18.0,301.0,Other,False,42.091202,192.1,2020-07-28,BLACK,-87.799004,Lexus,7.0,GX,38495.0,2532,4.551724,Fields Lexus Glenview,329.00,A,6-Speed Automatic,All-Wheel Drive,109.8,74.2,2017,16.5,75.80,10.65,8
4,Gasoline,SUV / Crossover,Durham,17.0,51,27713,3000.0,V6,White,True,24.6,69.40,24.0,241.0,Black,False,35.899101,189.8,2020-07-21,WHITE,-78.941299,Jeep,5.0,Grand Cherokee,17988.0,1053,4.180000,Hendrick Chevrolet Buick GMC Cadillac Southpoint,390.00,A,Automatic,Four-Wheel Drive,114.8,84.8,2014,20.5,78.90,11.82,11




---



# **Linear Regression**

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import time
import warnings
from tqdm import tqdm
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

# Ignore warnings
warnings.filterwarnings('ignore')

# Starting to track overall runtime
start_time = time.time()

with tqdm(total=6, desc="Processing and Training") as pbar:

    df_sample = df.sample(fraction=0.2, seed=42)  # Randomly sampling 20% of the data
    pbar.update(1)

    # Handling categorical columns
    cat_columns = [field for (field, dtype) in df_sample.dtypes.items() if dtype == "string"]
    stages = []
    for col_name in cat_columns:
        indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_indexed", handleInvalid="keep")
        encoder = OneHotEncoder(inputCols=[f"{col_name}_indexed"], outputCols=[f"{col_name}_encoded"])
        stages += [indexer, encoder]
    pbar.update(1)

    # Converting 'franchise_dealer' to numeric
    df_sample = df_sample.withColumn("franchise_dealer", F.col("franchise_dealer").cast("int"))

    # Assembling features
    num_columns = [col for col in df_sample.columns if col != 'price' and col not in cat_columns]
    encoded_columns = [f"{col}_encoded" for col in cat_columns]
    feature_columns = num_columns + encoded_columns
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    stages += [assembler]
    pbar.update(1)

    # Adding scaling to the pipeline
    scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
    stages += [scaler]

    # Creating and applying the pipeline
    pipeline = Pipeline(stages=stages)
    pipeline_model = pipeline.fit(df_sample)
    df_sample = pipeline_model.transform(df_sample)
    pbar.update(1)

    # Splitting the data
    train_df, test_df = df_sample.randomSplit([0.8, 0.2], seed=42)
    pbar.update(1)

    # Using Linear Regression
    lr = LinearRegression(featuresCol="scaled_features", labelCol="price")
    model = lr.fit(train_df)
    pbar.update(1)

# Making predictions
print("Making predictions...")
predictions = model.transform(test_df)

# Evaluating the model
print("Evaluating the model...")
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

# Displaying results
print(f"\nTrain size: {train_df.count():,} samples")
print(f"Test size: {test_df.count():,} samples")

# Multiplying R-Squared by 100 and rounding it off
print(f"\n\nR-Squared Score (Accuracy): {round(r2 * 100)}%")

# Calculating total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Processing and Training: 100%|██████████| 6/6 [7:04:58<00:00, 4249.79s/it]
Making predictions...
Evaluating the model...

Train size: 480,538 samples
Test size: 120,299 samples


R-Squared Score (Accuracy): 77.52%

Overall runtime: 428 minutes.


In [ ]:
# Calculating additional metrics
mae_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(predictions)

mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")
mse = mse_evaluator.evaluate(predictions)

rmse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = rmse_evaluator.evaluate(predictions)

print("Additional Metrics:")
print(f"Mean Absolute Error: {round(mae)}")
print(f"Mean Squared Error: {round(mse)}")
print(f"Root Mean Squared Error: {round(rmse)}")

Additional Metrics:
Mean Absolute Error: 4284
Mean Squared Error: 84093346
Root Mean Squared Error: 9170


The Mean Absolute Error (`MAE`) of **\$4284**  suggests that, on average, the predicted car prices deviate from the actual prices by this amount. Given that the Mean car price is **\$29933** , this error represents about `14.3% of the mean price`, which suggests that the model performs well.



In [ ]:
'''

# Define a different path where you want to save the model
new_model_path = "/content/drive/MyDrive/Big Data Analytics - Project/models/linear_regression_model_v2"

# Save the trained model to the new path
model.save(new_model_path)

print(f"Model saved successfully at {new_model_path}")

'''


Model saved successfully at /content/drive/MyDrive/Big Data Analytics - Project/models/linear_regression_model_v2


In [ ]:
features_list = pipeline_model.stages[-2].getInputCols()  # Getting feature names from VectorAssembler

coefficients = model.coefficients  # Getting the coefficients from the linear regression model

# Sorting features by the absolute value of their coefficients in descending order
sorted_features = [feature for feature, coefficient in sorted(zip(features_list, coefficients), key=lambda x: abs(x[1]), reverse=True)]

# Printing the ranked features from highest to lowest
print("Top 10 Features Ranked by Coefficient Magnitude (Highest to Lowest):")
for rank, feature in enumerate(sorted_features[:10], 1):  # Limiting to top 10 features
    print(f"{rank}. {feature}")


Top 10 Features Ranked by Coefficient Magnitude (Highest to Lowest):
1. horsepower
2. year
3. wheelbase
4. length
5. engine_displacement
6. is_new
7. width
8. listing_color_encoded
9. major_options_count
10. make_name_encoded




---



# **Predictions AFTER Feature Engineering**

In [ ]:
!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Feature_Engineered_DF.parquet' /content/

output_path = '/content/Feature_Engineered_DF.parquet'
df = spark.read.parquet(output_path)
print("The Feature Engineered DataFrame has been loaded successfully.")


The Feature Engineered DataFrame has been loaded successfully.


In [ ]:
# Printing the shape of the DataFrame
total_rows = df.count()
total_columns = len(df.columns)

print(f"The shape of the loaded DataFrame is: ({total_rows}, {total_columns})")

The shape of the loaded DataFrame is: (3000040, 47)


In [ ]:
# Calculating the average price
avg_price = df.agg({"price": "avg"}).collect()[0][0]
print(f"Average price of a car: {round(avg_price)}")

Average price of a car: 29933


In [ ]:
import pandas as pd
from IPython.display import display
import pyspark.sql.functions as F

# Converting the Spark DataFrame to a Pandas DataFrame and displaying 5 random rows with all columns
pd.set_option('display.max_columns', None)
pandas_df = df.orderBy(F.rand()).limit(5).toPandas()
display(pandas_df)


,fuel_type,body_type,city,city_fuel_economy,days_in_market,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,fuel_tank_volume,height,highway_fuel_economy,horsepower,interior_color,is_new,latitude,length,listing_color,longitude,make_name,maximum_seating,model_name,price,savings_amount,seller_rating,sp_name,torque,transmission,transmission_display,wheel_system_display,wheelbase,width,manufactured_year,combined_fuel_economy,legroom,log_mileage,major_options_count,hp_x_engine_disp,hp_x_torque,listed_day,listed_month,listed_year,age,resale_value_score,maintenance_cost,luxury_score
0,Gasoline,SUV / Crossover,Monroeville,25.000000,40,15146,2500.0,I4,Orange,True,14.5,68.4,32.000000,170.0,Gray,False,40.438702,184.5,ORANGE,-79.748802,Nissan,5.0,Rogue,21157.0,150,4.333333,#1 Cochran Hyundai of Monroeville,175.0,CVT,Continuously Variable Transmission,All-Wheel Drive,106.5,72.4,2017,28.50,80.9,10.74,8,0.30,0.73244,1,8,2020,3,18,34,29
1,Gasoline,SUV / Crossover,Greenville,27.000000,82,29607,1500.0,I4,White,True,14.0,66.5,32.000000,190.0,White,True,34.833698,182.1,GRAY,-82.341103,Honda,5.0,CR-V,35870.0,0,4.529412,Breakaway Honda,179.0,CVT,Continuously Variable Transmission,All-Wheel Drive,104.7,73.0,2020,29.50,81.7,8.91,6,0.72,0.51955,20,6,2020,0,22,39,31
2,Gasoline,Pickup Truck,Coraopolis,22.690001,188,15108,3500.0,V6,Red,True,26.0,77.2,29.469999,375.0,Gray,True,40.527100,231.9,RED,-80.210800,Ford,6.0,F-150,45508.0,0,4.166667,Moon Township Ford,400.0,A,Automatic,Four-Wheel Drive,145.0,96.8,2020,26.08,87.5,1.10,9,0.62,1.79666,6,3,2020,0,30,43,36
3,Gasoline,SUV / Crossover,O Fallon,26.000000,170,62269,2500.0,I4,Silver,True,14.5,68.0,33.000000,170.0,Black,True,38.592800,184.5,SILVER,-89.958900,Nissan,5.0,Rogue,27242.0,0,4.078948,Auffenberg Automall,175.0,CVT,Continuously Variable Transmission,Front-Wheel Drive,106.5,72.4,2020,29.50,80.9,2.48,3,0.30,0.73244,25,3,2020,0,25,37,30
4,Gasoline,Sedan,Stafford,31.000000,26,22554,2000.0,I4,Black,False,12.4,55.7,40.000000,158.0,Black,False,38.507801,182.3,BLACK,-77.371597,Honda,5.0,Civic,16977.0,455,4.360000,Off Lease Motors,138.0,CVT,Continuously Variable Transmission,Front-Wheel Drive,106.3,70.8,2017,35.50,79.7,10.23,4,0.73,1.19255,15,8,2020,3,23,34,28


In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import time
import warnings
from tqdm import tqdm
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

# Ignore warnings
warnings.filterwarnings('ignore')

# Starting to track overall runtime
start_time = time.time()

with tqdm(total=6, desc="Processing and Training") as pbar:

    df_sample = df.sample(fraction=0.2, seed=42)  # Randomly sampling 20% of the data
    pbar.update(1)

    # Handling categorical columns
    cat_columns = [field for (field, dtype) in df_sample.dtypes.items() if dtype == "string"]
    stages = []
    for col_name in cat_columns:
        indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_indexed", handleInvalid="keep")
        encoder = OneHotEncoder(inputCols=[f"{col_name}_indexed"], outputCols=[f"{col_name}_encoded"])
        stages += [indexer, encoder]
    pbar.update(1)

    # Converting 'franchise_dealer' to numeric
    df_sample = df_sample.withColumn("franchise_dealer", F.col("franchise_dealer").cast("int"))

    # Assembling features
    num_columns = [col for col in df_sample.columns if col != 'price' and col not in cat_columns]
    encoded_columns = [f"{col}_encoded" for col in cat_columns]
    feature_columns = num_columns + encoded_columns
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    stages += [assembler]
    pbar.update(1)

    # Adding scaling to the pipeline
    scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
    stages += [scaler]

    # Creating and applying the pipeline
    pipeline = Pipeline(stages=stages)
    pipeline_model = pipeline.fit(df_sample)
    df_sample = pipeline_model.transform(df_sample)
    pbar.update(1)

    # Splitting the data
    train_df, test_df = df_sample.randomSplit([0.8, 0.2], seed=42)
    pbar.update(1)

    # Using Linear Regression
    lr = LinearRegression(featuresCol="scaled_features", labelCol="price")
    model = lr.fit(train_df)
    pbar.update(1)

# Making predictions
print("Making predictions...")
predictions = model.transform(test_df)

# Evaluating the model
print("Evaluating the model...")
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

# Displaying results
print(f"\nTrain size: {train_df.count():,} samples")
print(f"Test size: {test_df.count():,} samples")

# Multiplying R-Squared by 100 and rounding it off
print(f"\n\nR-Squared Score (Accuracy): {round(r2 * 100)}%")

# Calculating total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Processing and Training: 100%|██████████| 6/6 [17:03:52<00:00, 10238.69s/it]


Making predictions...
Evaluating the model...

Train size: 480,411 samples
Test size: 120,366 samples


R-Squared Score (Accuracy): 84.20%

Overall runtime: 1039 minutes.


In [ ]:
new_model_path = "/content/drive/MyDrive/Big Data Analytics - Project/models/Linear_Regression_model_FE"

# Saving the trained model to the new path
model.save(new_model_path)

print(f"Model saved successfully at {new_model_path}")

Model saved successfully at /content/drive/MyDrive/Big Data Analytics - Project/models/Linear_Regression_model_FE


In [ ]:
# Calculating additional metrics
mae_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(predictions)

mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")
mse = mse_evaluator.evaluate(predictions)

rmse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = rmse_evaluator.evaluate(predictions)


print("Additional Metrics:")
print(f"Mean Absolute Error: {round(mae)}")
print(f"Mean Squared Error: {round(mse)}")
print(f"Root Mean Squared Error: {round(rmse)}")

Additional Metrics:
Mean Absolute Error: 4131
Mean Squared Error: 52079104
Root Mean Squared Error: 7217


In **Linear Regression**, the model saw an **improvement in R² accuracy by 6.7%**, while the Mean Absolute Error **(MAE) was reduced by **0.50%****, and the Root Mean Square Error **(RMSE) was reduced by **6.52%****, demonstrating enhanced predictive performance.

In [ ]:
features_list = pipeline_model.stages[-2].getInputCols()  # Getting feature names from VectorAssembler

coefficients = model.coefficients  # Getting the coefficients from the linear regression model

# Sorting features by the absolute value of their coefficients in descending order
sorted_features = [feature for feature, coefficient in sorted(zip(features_list, coefficients), key=lambda x: abs(x[1]), reverse=True)]

# Printing the ranked features from highest to lowest
print("Top 10 Features Ranked by Coefficient Magnitude (Highest to Lowest):")
for rank, feature in enumerate(sorted_features[:10], 1):  # Limiting to top 10 features
    print(f"{rank}. {feature}")


Top 10 Features Ranked by Coefficient Magnitude (Highest to Lowest):
1. horsepower
2. maintenance_cost
3. engine_displacement
4. is_new
5. hp_x_torque
6. luxury_score
7. resale_value_score
8. torque
9. width
10. wheelbase




---

